In [2]:
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [3]:
with open('key/sk_open_api_key.txt') as f_:
    sk_key = f_.read()

In [4]:
len(sk_key)

40

In [25]:
# 혼잡도
station_code = 133
dow = 'MON' # 대문자
hh = '08'
url = f"https://apis.openapi.sk.com/puzzle/subway/congestion/stat/train/stations/{station_code}?dow={dow}&hh={hh}"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "appkey": sk_key
}

response = requests.get(url, headers=headers).json()


In [26]:
response.keys()

dict_keys(['status', 'contents'])

In [27]:
response['contents'].keys()

dict_keys(['subwayLine', 'stationName', 'stationCode', 'stat', 'statStartDate', 'statEndDate'])

In [32]:
len(response['contents'])

6

In [40]:
response['contents']['stat'][0]['startStationName']

'소요산역'

In [46]:
response['contents']['stationName']

'서울역'

In [146]:
type(response['contents']['stat'][55]['data'][0])

dict

In [141]:
temp_list = []
temp_list2 = []
for stat in response['contents']['stat']:
    if stat['updnLine'] == 1:
        temp_list.append(stat['data'])
    elif stat['updnLine'] == 0:
        temp_list2.append(stat['data'])
# temp_list = temp_list[0]
# temp_list
# temp_list2 = temp_list2[0]
# temp_list2


In [142]:
data1 = [stat['data'] for stat in response['contents']['stat'] if stat['updnLine'] == 1]
data2 = [stat['data'] for stat in response['contents']['stat'] if stat['updnLine'] == 0]

In [143]:
data1

[[{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 23},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 47},
  {'dow': 'MON', 'hh': '08', 'mm': '30', 'congestionTrain': 47},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 23},
  {'dow': 'MON', 'hh': '08', 'mm': '50', 'congestionTrain': 0}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 63},
  {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 63},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 63},
  {'dow': 'MON', 'hh': '08', 'mm': '30', 'congestionTrain': 51},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 38},
  {'dow': 'MON', 'hh': '08', 'mm': '50', 'congestionTrain': 38}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08',

In [140]:
temp_list

[[{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 47},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 23}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 63},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 63},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 38}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 0}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 21},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 43}],
 [{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 57},
  {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 55},
  {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 54}],
 [{'dow': 'MON', 'hh': '0

In [116]:
df = pd.DataFrame(temp_list)
df['updn'] = 1

In [123]:
df1 = pd.DataFrame(temp_list)
df1['updn'] = 1
df1.drop(columns=['dow','hh','mm'],inplace=True)
df1 = df1.groupby('updn')['congestionTrain'].agg('mean').reset_index()
df_ = pd.DataFrame(temp_list2)
df_['updn'] = 0
df_.drop(columns=['dow','hh','mm'],inplace=True)
df_ = df_.groupby('updn')['congestionTrain'].agg('mean').reset_index()
df_res = pd.concat([df1,df_])
df_res


,updn,congestionTrain
0,1,23.333333
0,0,0.000000


In [113]:
df = pd.DataFrame(temp_list)


In [61]:
a = response['contents']['stat'][0]['data']

In [90]:
df = pd.DataFrame(a)

In [117]:
df

,dow,hh,mm,congestionTrain,updn
0,MON,08,00,0,1
1,MON,08,10,23,1
2,MON,08,20,47,1
3,MON,08,30,47,1
4,MON,08,40,23,1
5,MON,08,50,0,1


In [124]:
df1 = df[df['congestionTrain'] != 0].copy()
df1.drop(columns=['dow','hh','mm'],inplace=True)
df1

,congestionTrain
1,23
2,47
3,47
4,23


In [119]:
df1 = df1.groupby('updn')['congestionTrain'].agg('mean').reset_index()

In [120]:
df1

,updn,congestionTrain
0,1,35.0


In [126]:
df = pd.DataFrame(temp_list)
df['updn'] = 1
df1 = df[df['congestionTrain'] != 0].copy()
df1.drop(columns=['dow','hh','mm'],inplace=True)
df1 = df1.groupby('updn')['congestionTrain'].agg('mean').reset_index()
df = pd.DataFrame(temp_list2)
df['updn'] = 0
df2 = df[df['congestionTrain'] != 0].copy()
df2.drop(columns=['dow','hh','mm'],inplace=True)
df2 = df2.groupby('updn')['congestionTrain'].agg('mean').reset_index()

In [128]:
temp_list2

[{'dow': 'MON', 'hh': '08', 'mm': '00', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '10', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '20', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '30', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '40', 'congestionTrain': 0},
 {'dow': 'MON', 'hh': '08', 'mm': '50', 'congestionTrain': 0}]